In [1]:
# mysql connection
import mysql.connector

mydb = mysql.connector.connect(
  host="cowstudio.wayne-lee.cn",
  user="cowstudio",
  password="cowstudio_2119",
  database="cowstudio"
)

In [2]:
# get spark session, 2g mem per executor
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import os

# set python env
os.environ['PYSPARK_PYTHON'] = "/opt/conda3/envs/lab2/bin/python"
spark = SparkSession.builder \
    .appName("item_order_list") \
    .master("spark://node01:10077") \
    .enableHiveSupport()\
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.cores.max", "3") \
    .config("spark.sql.shuffle.partitions", "12") \
    .config("spark.sql.autoBroadcastJoinThreshold", "-1") \
    .getOrCreate()

sc = spark.sparkContext

23/04/22 08:20:17 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/22 08:20:19 WARN spark.SparkContext: Please ensure that the number of slots available on your executors is limited by the number of cores to task cpus and not another custom resource. If cores is not the limiting resource then dynamic allocation will not work properly!


In [3]:
# define map functions 
from datetime import datetime, timedelta

today_string = datetime.today().strftime('%Y-%m-%d')
print(today_string)

fresh_list_thr = 10
host_list_size = 30

2023-04-22


In [4]:
item_order = spark.sql(f'''
with item_all as(
    select
        id as item_id,
        category
    from
        item_ods
)
select
    item_all.item_id,
    item_all.category,
    item_score.score,
    row_number() over(
        partition by
            category
        order by
            score desc
    ) as rn
from
    item_all
left join
    item_score on item_all.item_id = item_score.item_id
where
    date = '{today_string}'
order by
    rn
''')
item_order.show()
item_order.createOrReplaceTempView("item_order")

+-------+---------------+-----+---+
|item_id|       category|score| rn|
+-------+---------------+-----+---+
|    455|service_product| 35.0|  1|
|    198|       breeding| 14.0|  1|
|    461|        service| 34.0|  1|
|     95| cattle_product| 68.0|  1|
|    364|   whole_cattle| 34.0|  1|
|    232|service_product| 20.0|  2|
|    191|       breeding| 10.0|  2|
|    359|   whole_cattle| 25.0|  2|
|     77| cattle_product| 37.0|  2|
|    449|        service| 34.0|  2|
|    121|   whole_cattle|  7.0|  3|
|    197|       breeding| 10.0|  3|
|    454|        service| 21.0|  3|
|    221|service_product| 20.0|  3|
|     78| cattle_product| 32.0|  3|
|    457|        service| 20.0|  4|
|    212|service_product| 16.0|  4|
|    203|       breeding|  8.0|  4|
|    343| cattle_product| 30.0|  4|
|    355|   whole_cattle|  7.0|  4|
+-------+---------------+-----+---+
only showing top 20 rows



In [5]:
item_fresh_list = spark.sql(f'''
select
    *
from
    item_order
where
    score < {fresh_list_thr}
''')
item_fresh_list.show()
item_fresh_list.createOrReplaceTempView("item_fresh_list")

+-------+---------------+-----+---+
|item_id|       category|score| rn|
+-------+---------------+-----+---+
|    121|   whole_cattle|  7.0|  3|
|    355|   whole_cattle|  7.0|  4|
|    203|       breeding|  8.0|  4|
|    196|       breeding|  8.0|  5|
|    361|   whole_cattle|  7.0|  5|
|    403|   whole_cattle|  6.0|  6|
|    452|        service|  4.0|  6|
|    194|       breeding|  5.0|  6|
|    223|service_product|  8.0|  6|
|    372|       breeding|  4.0|  7|
|    257|service_product|  8.0|  7|
|    389|        service|  4.0|  7|
|    414|   whole_cattle|  5.0|  7|
|    456|        service|  4.0|  8|
|    220|service_product|  6.0|  8|
|    127|   whole_cattle|  4.0|  8|
|    195|       breeding|  2.0|  8|
|    298|        service|  2.0|  9|
|    230|service_product|  5.0|  9|
|    193|       breeding|  2.0|  9|
+-------+---------------+-----+---+
only showing top 20 rows



In [6]:
item_hot_list = spark.sql(f'''
select
    a.*
from
    item_order a
left join
    item_fresh_list b on a.item_id = b.item_id 
where
    a.rn <= {host_list_size}
and
    b.item_id is null
''')
item_hot_list.show()
item_hot_list.createOrReplaceTempView("item_hot_list")

+-------+---------------+-----+---+
|item_id|       category|score| rn|
+-------+---------------+-----+---+
|    232|service_product| 20.0|  2|
|    343| cattle_product| 30.0|  4|
|    461|        service| 34.0|  1|
|     77| cattle_product| 37.0|  2|
|    191|       breeding| 10.0|  2|
|    197|       breeding| 10.0|  3|
|     41| cattle_product| 20.0|  8|
|     80| cattle_product| 13.0|  9|
|     87| cattle_product| 12.0| 11|
|    359|   whole_cattle| 25.0|  2|
|    449|        service| 34.0|  2|
|     83| cattle_product| 10.0| 13|
|    198|       breeding| 14.0|  1|
|    221|service_product| 20.0|  3|
|    455|service_product| 35.0|  1|
|    457|        service| 20.0|  4|
|     81| cattle_product| 11.0| 12|
|    212|service_product| 16.0|  4|
|    228|service_product| 15.0|  5|
|    364|   whole_cattle| 34.0|  1|
+-------+---------------+-----+---+
only showing top 20 rows



In [7]:
# spark.sql("drop table if exists ").show()

In [10]:
item_hot_list.write.mode("overwrite").partitionBy("date").s.partitionBy("date")aveAsTable("item_hot_list")

In [10]:
item_hot_list.write.mode("overwrite").partitionBy("date").s.partitionBy("date")aveAsTable("item_hot_list")

In [10]:
item_hot_list.write.mode("overwrite").partitionBy("date").s.partitionBy("date")aveAsTable("item_hot_list")

In [11]:
spark.sql("show tables").show()

+--------+---------------+-----------+
|database|      tableName|isTemporary|
+--------+---------------+-----------+
| default|      event_ods|      false|
| default|item_fresh_list|      false|
| default|  item_hot_list|      false|
| default|       item_ods|      false|
| default|     item_order|      false|
| default|     item_score|      false|
| default|   item_tag_ods|      false|
| default|item_word_count|      false|
| default|  item_word_idf|      false|
| default|   item_word_tf|      false|
| default|item_word_tfidf|      false|
| default|        tag_ods|      false|
| default|           test|      false|
| default|          test2|      false|
| default|       user_ods|      false|
| default|   user_tag_ods|      false|
|        |item_fresh_list|       true|
|        |  item_hot_list|       true|
|        |     item_order|       true|
+--------+---------------+-----------+



In [12]:
spark.stop()
mydb.close()